In [1]:
from fosforml.model_manager.snowflakesession import get_session

In [2]:
my_session = get_session()

In [3]:
table_name = 'Fact_Customer_Orders'
sf_df = my_session.sql("select * from {}".format(table_name)) 
 
df = sf_df.to_pandas() 
print(df.shape)

(225563, 4)


In [4]:
df.columns

Index(['ORDER_ID', 'PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE'], dtype='object')

In [5]:
df.head(5)

,ORDER_ID,PRODUCT_ID,ORDER_QTY,ORDER_DATE
0,Order_000006,Product_001,850,2019-01-02
1,Order_000001,Product_004,700,2019-01-02
2,Order_000003,Product_004,900,2019-01-02
3,Order_000008,Product_004,550,2019-01-02
4,Order_000012,Product_004,300,2019-01-02
